<a href="https://colab.research.google.com/github/TharinsaMudalige/Neuron-Brain_Tumor_Detection_Classification_with_XAI/blob/Detection-Classification-VIT/Tumor_Segmentation_using_U_NET_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')


Load Datasets

In [2]:
# Paths to image and mask folders
IMAGES_PATH = '/content/drive/MyDrive/DSGP_BrainTumorDetection/Segmentation Task/archive (3)/images'
MASKS_PATH = '/content/drive/MyDrive/DSGP_BrainTumorDetection/Segmentation Task/archive (3)/masks'

In [3]:
# Hyperparameters
SIZE = 128  # Reduced size to optimize memory usage
CHANNEL = 1
Num_Of_Classes = 1
smooth = 1e-15

Read Datasets

In [4]:
# Read and sort image/mask files
image_files = sorted(os.listdir(IMAGES_PATH))
mask_files = sorted(os.listdir(MASKS_PATH))

In [7]:
# Debug: Ensure image and mask counts match
assert len(image_files) == len(mask_files), "Number of images and masks do not match!"

In [6]:
# Function to display a few images and masks
def display_images_masks(image_files, mask_files, imagepath, maskpath):
    for count, (image_file, mask_file) in enumerate(zip(image_files, mask_files)):
        if count >= 4:  # Display only 4 pairs
            break
        imagepath_full = os.path.join(imagepath, image_file)
        maskpath_full = os.path.join(maskpath, mask_file)

        image = cv2.imread(imagepath_full)
        mask = cv2.imread(maskpath_full)

        if image is None or mask is None:
            raise ValueError(f"Image or mask at path {imagepath_full} or {maskpath_full} could not be loaded")

        plt.figure(figsize=(10, 10))

        # Display the image
        plt.subplot(1, 3, 1)
        plt.imshow(image, cmap='gray')
        plt.title('Image')

        # Display the mask
        plt.subplot(1, 3, 2)
        plt.imshow(mask, cmap='gray')
        plt.title('Mask')

        # Display contours
        plt.subplot(1, 3, 3)
        mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        contours, _ = cv2.findContours(mask_gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        overlay = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), thickness=2)
        plt.imshow(overlay)
        plt.title('Image with Contours')

        plt.show()

Pre Processing images and masks

In [8]:
# Preprocessing images and masks
def preprocessing_images_masks(image_files, mask_files, imagepath, maskpath):
    images, masks = [], []
    for image_file, mask_file in tqdm(zip(image_files, mask_files)):
        imagepath_full = os.path.join(imagepath, image_file)
        maskpath_full = os.path.join(maskpath, mask_file)

        # Preprocess images
        image = cv2.imread(imagepath_full, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (SIZE, SIZE))
        image = image / 255.0
        images.append(image)

        # Preprocess masks
        mask = cv2.imread(maskpath_full, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (SIZE, SIZE))
        mask = mask / 255.0
        masks.append(mask)

    return np.expand_dims(np.array(images), axis=-1), np.expand_dims(np.array(masks), axis=-1)

Dice Coefficient and loss functions

In [ ]:
# Dice coefficient and loss functions
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

U-Net model

In [9]:
# U-Net model definition
def unet_model(input_size=(SIZE, SIZE, CHANNEL), num_classes=Num_Of_Classes):
    inputs = layers.Input(input_size)

    # Encoding path
    c1 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    # Decoding path
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = layers.Conv2D(num_classes, (1, 1), activation='sigmoid')(c9)

    return models.Model(inputs=[inputs], outputs=[outputs])